<a href="https://colab.research.google.com/github/Oec8cek/Ai-Understanding/blob/main/%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88%EB%89%B4%EC%8A%A4_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로컬(주피터)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install webdriver_manager

# 코랩

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [5]:
pip install chromedriver_autoinstaller

In [163]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import random

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import os
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# Dataframe
import pandas as pd

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)

# 잠깐!

In [9]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import subprocess

from datetime import datetime

def get_news_url(query, start_date, end_date, page=1):
    # 입력된 날짜를 YYYY.MM.DD 형식으로 변환
    start_date_formatted = datetime.strptime(start_date, "%Y-%m-%d").strftime("%Y.%m.%d")
    end_date_formatted = datetime.strptime(end_date, "%Y-%m-%d").strftime("%Y.%m.%d")

    base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=0&field=0&pd=3&cluster_rank=20&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{}to{},a:all&start={}'

    formatted_url = base_url.format(query, start_date_formatted, end_date_formatted, (page - 1) * 10 + 1)
    return formatted_url

In [10]:
def crawl_naver_news(query, start_date, end_date):
    news_dict = {}
    idx = 0
    cur_page = 1

    while True:
        url = get_news_url(query, start_date, end_date, page=cur_page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126)
        table = soup.find('ul', {'class': 'list_news'})
        li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
        area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
        a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

        if not a_list:
            break  # 뉴스 기사가 더 이상 없으면 종료

        for n in a_list:
            news_dict[idx] = {'title': n.get('title'), 'url': n.get('href')}
            idx += 1

        cur_page += 1

    return news_dict

# 빅카인즈 기업 검색

In [11]:
# 결과를 저장할 데이터프레임 초기화
results_df = pd.DataFrame(columns=['Company', 'Industry'])

In [12]:
def get_page_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    cards = soup.find_all('div', class_='card')

    page_data = []
    for card in cards:
        company_name = card.find('p', {'class': 'k-name p-ellipsis'}).text
        industry = card.find('dd', {'class': 'p-ellipsis'}).text
        page_data.append({'Company': company_name, 'Industry': industry})

    return page_data

In [13]:
data = []
start_page=1

In [14]:
while True:
    # 웹 페이지 열기
    url = f"https://www.bigkinds.or.kr/v2/depthAnalysis/company.do?codeGroupId=corp_top300_list&page={start_page}"
    driver.get(url)

    # 게시글 테이블 로딩을 기다림
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-wrap.cl div.card")))

    # 현재 페이지에서 회사 정보 수집
    cards = driver.find_elements(By.CSS_SELECTOR, "div.card-wrap.cl div.card")
    for card in cards:
        company_name_kr = card.find_element(By.CSS_SELECTOR, "p.k-name.p-ellipsis").text.strip()
        company_name_en = card.find_element(By.CSS_SELECTOR, "p.e-name.p-ellipsis").text.strip()
        industry = card.find_element(By.CSS_SELECTOR, "dd.p-ellipsis").text.strip()
        representative = card.find_elements(By.TAG_NAME, "dd")[-1].text.strip()

        # 회사 정보를 딕셔너리에 저장
        company_info = {
            "한글 회사명": company_name_kr,
            "영문 회사명": company_name_en,
            "업종": industry,
            "대표이름": representative
        }

        # 딕셔너리를 리스트에 추가
        data.append(company_info)

    start_page += 1

    if start_page > 19:
      break

results = pd.DataFrame(data)
results

,한글 회사명,영문 회사명,업종,대표이름
0,삼성전자,"Samsung Electronics Co., Ltd.",통신 및 방송 장비 제조업,"한종희, 경계현"
1,현대자동차,Hyundai Motor Company,승용차 및 기타 여객용 자동차 제조업,"정의선, 장재훈, 이동석(각자 대표이사)"
2,기아,Kia Corporation,자동차 제조업,"송호성, 최준영"
3,LG전자,LG Electronics Inc.,통신 및 방송 장비 제조업,"조주완, 배두용"
4,한국전력공사,KOREA ELECTRIC POWER CORPORATION,송전 및 배전업,정승일
...,...,...,...,...
295,셀트리온,"Celltrion Co., Ltd.",생물학적 제제 제조업,기우성
296,아우디폭스바겐코리아,Audi Volkswagen Korea Ltd.,자동차 신품 판매업,미하엘안드레아막스프레드리히프리시
297,아이에스동서,"IS Dongseo Co., Ltd.",타일 및 유사 비내화 요업제품 제조업,권민석
298,한화자산운용,"Hanwha Asset Management Co., Ltd.",신탁업 및 집합 투자업,권희백


In [15]:
# 데이터프레임 저장
company = results
company.to_csv('company_list.csv', index=False)

In [16]:
company_list = list(company['한글 회사명'])

In [17]:
company

,한글 회사명,영문 회사명,업종,대표이름
0,삼성전자,"Samsung Electronics Co., Ltd.",통신 및 방송 장비 제조업,"한종희, 경계현"
1,현대자동차,Hyundai Motor Company,승용차 및 기타 여객용 자동차 제조업,"정의선, 장재훈, 이동석(각자 대표이사)"
2,기아,Kia Corporation,자동차 제조업,"송호성, 최준영"
3,LG전자,LG Electronics Inc.,통신 및 방송 장비 제조업,"조주완, 배두용"
4,한국전력공사,KOREA ELECTRIC POWER CORPORATION,송전 및 배전업,정승일
...,...,...,...,...
295,셀트리온,"Celltrion Co., Ltd.",생물학적 제제 제조업,기우성
296,아우디폭스바겐코리아,Audi Volkswagen Korea Ltd.,자동차 신품 판매업,미하엘안드레아막스프레드리히프리시
297,아이에스동서,"IS Dongseo Co., Ltd.",타일 및 유사 비내화 요업제품 제조업,권민석
298,한화자산운용,"Hanwha Asset Management Co., Ltd.",신탁업 및 집합 투자업,권희백


In [161]:
num_company = len(company_list)
num_company

300

# 빅카인즈X 네이버 기사 가져오기

In [109]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm

# 모든 페이지의 기사를 저장할 리스트
all_articles = []

# 시작 페이지 번호
start_page = 1
results_per_page = 10           # 이거 그냥 결과만 빠르게 보고 싶어서 임의로 설정. 실제론 그때그때 다름.

In [20]:
2# 네이버 검색 API 호출 URL
api_url = "https://openapi.naver.com/v1/search/news.json"

client_id = 'Zwdz_lbupo91QAvrtqsz'
client_secret = 'ifGZTCfREm'

# API 호출 시 사용할 헤더 정보 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

In [160]:
def parse_date(date):
    # 입력값을 문자열로 변환하여 앞에서부터 4자리는 year, 4자리는 month, 나머지는 date로 추출
    date_str = str(date)
    year = date_str[:4]
    month = date_str[4:6]
    day = date_str[6:]
    r =  f'{year}.{month}.{day}.'
    # .으로 구분하여 반환
    return str(r)


# 날짜 파싱을 위한 함수
#def parse_date(date):
#    return datetime.strptime(str(date), '%Y.%m.%d.').strftime('%Y%m%d')

## 기간 설정..

In [168]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

# 원하는 기업 찾기
#query = input('검색 키워드를 입력하세요 : ')
# 랜덤하게 기업 검색해보자
rand = random.randint(1, num_company)
query = company_list[rand]
query = query.replace(' ', '+')

print(query+' 기사 검색 중')

# 날짜 입력 받기
# 일단 예시로 한달 단위만
#start_date = input('시작 날짜를 입력하세요 (예:20010717): ')
#end_date = input('종료 날짜를 입력하세요 (예:20010717): ')
start_date = 20231126
end_date = 20231226

news_url_ = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={query}&sort=1'

한화손해보험 기사 검색 중


In [169]:
# 시작 날짜와 종료 날짜 설정 (YYYYMMDD 형식)
start_date = '20231201'
end_date = '20231225'

# 시작 페이지와 페이지당 결과 수 설정
start_page = 1

# 결과를 저장할 리스트
news_list = []

while True:
    start_value = (start_page - 1) * results_per_page + 1
    # 시작 날짜와 종료 날짜를 포함한 URL 생성
    news_url = f"{news_url_}&start={start_value}&ds={start_date}&de={end_date}"

    req = requests.get(news_url)

    # 요청이 성공적으로 이루어진 경우에만 진행
    if req.status_code == 200:
        soup = BeautifulSoup(req.text, 'html.parser')

        table = soup.find('ul', {'class': 'list_news'})
        li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
        area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
        a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

        # 이 부분을 고쳐보자
        for n in a_list:
            news_dict = {'title' : n.get('title'), 'url' : n.get('href') }
            news_list.append(news_dict)

        # 다음 페이지로 이동
        start_value = (start_page - 1) * results_per_page + 1
        start_page += 1
    else:
        # 요청이 실패하면 루프 종료
        break

# news_list에서 title과 url 값을 추출하여 리스트로 만듭니다.
titles = [news['title'] for news in news_list]
urls = [news['url'] for news in news_list]

# DataFrame을 생성합니다.
data = {'title': titles, 'url': urls}
sample = pd.DataFrame(data)
sample

,title,url
0,"'우리 사회 온도 높이는 기업'…한화그룹, 이웃사랑성금 40억 '쾌척'",https://www.thedailypost.kr/news/articleView.h...
1,한파로 전국 ‘꽁꽁’… 보험업계 “피해 최소화 노력”,http://www.insnews.co.kr/design_php/news_view....
2,"[인구절벽의 시대, 보험의 역할]보험업계도 ‘저출생’ 위기 극복에 힘 모아",http://www.insnews.co.kr/design_php/news_view....
3,[기자의 눈]여전히 두꺼운 보험업계 유리천장,http://www.insnews.co.kr/design_php/news_view....
4,“혹시 애 안 생기면 어떡해요”…20대 여성 4명 중 1명 ‘이것’ 가입했다,https://www.mk.co.kr/article/10906531
...,...,...
185,"한화그룹, 연말 이웃사랑성금 40억원 기부...김승연 회장 ""사회의 온도 높여야""",http://www.kdfnews.com/news/articleView.html?i...
186,"한화그룹, 연말 이웃사랑성금 40억원 전달",https://www.businesskorea.co.kr/news/articleVi...
187,보험사 배당 재개할까… 변수는 13조 해약준비금,https://biz.newdaily.co.kr/site/data/html/2023...
188,"""올 겨울 메서운 한파 잦다""...누수∙동파 대비 보험상품에 관심 UP",https://www.greened.kr/news/articleView.html?i...


In [94]:
url = news_url_
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")
soup_tit = soup.find_all("a", attrs={"class": "news_tit"})
soup_info = soup.find_all("span", attrs={"class": "info"})
for tit, info in zip(soup_tit, soup_info):
  print(f"{info.get_text()} {tit.get_text()} {tit['href']}")

25면 TOP 석유 없는 세계로 한 걸음 다가가는 인류... 그러나 아직 이르다 [세계사로 읽...  https://www.hankookilbo.com/News/Read/A2023122514380001323?did=NA
13분 전 1월로 밀린 새해 경제정책방향… 경기부양 해법 놓고 고민  https://news.kmib.co.kr/article/view.asp?arcid=0924336704&code=11151100&cp=nv
15면 1단 새해 50개국 20억명 투표장으로… 지구촌 경제·안보 요동친다  https://news.kmib.co.kr/article/view.asp?arcid=0924336662&code=11141100&cp=nv
34분 전 [경제시평] 금리 인하 뒤집어보기  https://news.kmib.co.kr/article/view.asp?arcid=0924336364&code=11171313&cp=nv
2면 TOP [해외 경제지표] 2023년 12월 25일자  https://www.chosun.com/economy/stock-finance/2023/12/26/LPZMLRAKVNBX3LUQGB6YYNYEGQ/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news
35분 전 “대기업-中企 화합해 위기 극복”… 경제계 500명 합동 신년인사회  https://www.donga.com/news/Economy/article/all/20231225/122772366/1
30면 1단 [화요경제 항산항심] 스타트업 경영자에게 배워야 할 것들  http://www.kookje.co.kr/news2011/asp/newsbody.asp?code=1700&key=20231226.22022007830
40분 전 “내년 주요국 물가 2% 안착 전망”… 50개국 선거가 경제에 변수  https://www.donga.com/news/Inter/article/all/20231226/122772700/1
1시간 전 [되

## 기사 개수 설정..

In [30]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

# 원하는 기업 찾기
query = input('검색 키워드를 입력하세요 : ')
query = query.replace(' ', '+')

# 기사 수 제한 두기 (아니면 기사수가 1000개 넘으면 1000개까지만 크롤링하고 총 기사 개수만 수집하기?)
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))

# url 생성
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')

news_dict = {}
idx = 0

print()
print('크롤링 중...')

while idx < news_num:
### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###

    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]

    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    start_page += 1

    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(start_page)][0].get('href')

    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T
news_df

시작 날짜를 입력하세요 (예:20010717): 20231224
종료 날짜를 입력하세요 (예:20010717): 20231225

크롤링 중...
크롤링 완료
데이터프레임 변환


,title,url
0,5년간 공공임대 거주 220명 극단 선택…영구임대 특히 많아(종합),http://www.kookje.co.kr/news2011/asp/newsbody....
1,"공학한림원, 신입 정회원 50명 선출",https://www.chosun.com/economy/science/2023/12...
2,"與, 네이버 혁신준비포럼에 “시간끌기용 안돼”",https://www.donga.com/news/Politics/article/al...
3,"[단독]“플랫폼기업 반칙행위, 매출액 10%까지 과징금 검토”",https://www.donga.com/news/Economy/article/all...
4,'불법 공매도' 역대 최대 과징금 · 검찰 고발,https://news.sbs.co.kr/news/endPage.do?news_id...
...,...,...
95,BNP파리바·HSBC 불법 공매도에 과징금 265억 원,https://www.ytn.co.kr/_ln/0102_202312251321158491
96,애플 'AI 뉴스 협상료 5000만 달러' 제안에 미온적인 이유,https://www.mediatoday.co.kr/news/articleView....
97,‘불법 공매도’ BNP파리바·HSBC 과징금 265억 원 ‘역대 최대’,https://www.viva100.com/main/view.php?key=2023...
98,지드래곤 마약 퇴치·중독치료 재단 설립 3억 기부,https://www.nocutnews.co.kr/news/6068458


In [ ]:

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
os.startfile(folder_path)

# 기사 본문 크롤링

In [21]:
####### 언론사별 본문 위치 태그 파싱 함수 ###########
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')

    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url):
        main_article = soup.find('div', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})

        text = main_article.text

    # MBC
    elif '//imnews.imbc' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text

    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text

    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text

    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text

    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text

    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text

    # 그 외
    else:
        text = None

    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')

In [ ]:
press_nm = 'KBS'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = 'C:/chromedriver/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:

    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)

    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]

    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))

    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))

        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)

        break



################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)

while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]

    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'),
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}

        idx += 1
        pbar.update(1)

    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()

        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break

#### 데이터 전처리하기 ######################################################

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

# 망한 코드...열지마세요,,,,,,,,,, 숨겨놓고 다시 짜기

In [ ]:
start_page = 1
news_dict = {}
idx = 0
total_news = 0  # 가져온 총 기사 개수
news_num = 1000

while True:
    start_value = (start_page - 1) * results_per_page + 1
    # url 생성
    news_url = news_url_ + '&start=' + str(start_value)
    #'&ds=' + parse_date(start_date) + '&de=' + parse_date(end_date)

    req = requests.get(news_url)

    # 요청이 성공적으로 이루어진 경우에만 진행
    if req.status_code == 200:
        soup = BeautifulSoup(req.text, 'html.parser')

        table = soup.find('ul', {'class': 'list_news'})
        li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
        area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
        a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

        # 이 부분을 고쳐보자
        for n in a_list[:min(len(a_list), news_num-idx)]:
            news_dict[idx] = {'title' : n.get('title'), 'url' : n.get('href') }
            idx += 1
            total_news += 1

            if total_news >= news_num:
              break


        # 다음 페이지로 이동
        results_per_page = 10  # 페이지당 결과 수
        start_page += 1
    else:
        # 요청이 실패하면 루프 종료
        break

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T
news_df


In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

query = input('검색 키워드를 입력하세요 : ')
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
query = query.replace(' ', '+')

news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')


news_dict = {}
idx = 0
cur_page = 1

while idx < news_num:
### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###

    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]

    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    cur_page += 1

    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')

    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T
news_df

검색 키워드를 입력하세요 : 삼성전자
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 100
크롤링 완료
데이터프레임 변환


,title,url
0,"갤럭시에서 '패스키' 쓴다…삼성전자, 원 UI 업데이트",https://www.yna.co.kr/view/AKR2023121901540001...
1,"'확장현실(XR) 열린다' 삼성전자, XR용 이미지센서 출시",https://www.etnews.com/20231219000194
2,박쥐처럼 감지하고 사람처럼 본다...진화한 삼성전자의 ‘눈’,http://news.heraldcorp.com/view.php?ud=2023121...
3,"삼성전자, '미래 첨단산업의 눈' 아이소셀 비전 2종 공개",http://www.newsis.com/view/?id=NISX20231219_00...
4,"""돈 앞에서도 의좋은 형제""…아우 이끄는 LG전자 VS 형님 돕는 삼성디스플레이",http://www.inews24.com/view/1667010
...,...,...
95,"""8만전자 가자~""…삼성전자, 52주 신고가 경신",https://www.hankyung.com/article/2023121454616
96,"""2나노, 판세 바뀐다""… 삼성전자 ‘TSMC 독주 막기’ 맹추격",http://www.fnnews.com/news/202312121808347115
97,"삼성전자, 삼성 TV 플러스에 영화 VOD 서비스 출시",https://www.yna.co.kr/view/AKR2023121302450000...
98,"[자막뉴스] 中 신생업체가 쥔 삼성전자 반도체 공정도...""피해 막대할 것""",https://www.ytn.co.kr/_ln/0134_202312181503472231


In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import os

# 사용자로부터 시작 날짜와 종료 날짜 입력 받기
start_date_str = input('시작 날짜를 입력하세요 (YYYY-MM-DD): ')
end_date_str = input('종료 날짜를 입력하세요 (YYYY-MM-DD): ')

# 날짜 형식을 확인하고 datetime으로 변환
try:
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
except ValueError:
    print('올바른 날짜 형식이 아닙니다. YYYY-MM-DD 형식으로 입력해주세요.')
    exit()

# 시작 날짜와 종료 날짜 사이의 날짜 목록 생성
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# 결과를 저장할 딕셔너리 초기화
news_dict = {}
idx = 0

for date in date_list:
    date_str = date.strftime('%Y-%m-%d')

    # 검색할 쿼리와 네이버 뉴스 URL 생성
    query = f'{date_str} {query}'  # 날짜를 검색 쿼리에 포함
    query = query.replace(' ', '+')
    news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}'

    req = requests.get(news_url)
    soup = BeautifulSoup(req.text, 'html.parser')

    # 네이버 뉴스 크롤링
    table = soup.find('ul', {'class': 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

    for n in a_list:
        news_dict[idx] = {'title': n.get('title'), 'url': n.get('href')}
        idx += 1

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T
news_df


시작 날짜를 입력하세요 (YYYY-MM-DD): 2023-12-01
종료 날짜를 입력하세요 (YYYY-MM-DD): 2023-12-15
크롤링 완료
데이터프레임 변환


,title,url
0,[Who Is ?] 황성우 삼성SDS 대표이사 사장,https://www.businesspost.co.kr/BP?command=arti...
1,[Who Is ?] 장덕현 삼성전기 대표이사 사장,https://www.businesspost.co.kr/BP?command=arti...
2,주간 코스피 기관 순매수도 상위 10종목...삼성전자-삼성전기-한화오션,http://www.betanews.net:8080/article/1453700.html
3,"[더벨][Financial Index/삼성그룹]호텔신라와 삼성전자, 극과 극의 차입...",https://www.thebell.co.kr/free/content/Article...
4,[Who Is ?] 장범식 숭실대학교 총장,https://www.businesspost.co.kr/BP?command=arti...
...,...,...
145,"'류이치 사카모토: 오퍼스' 거장의 마지막 연주, 미사여구가 필요하지 않았다 [TE...",https://tenasia.hankyung.com/article/202312082...
146,2023년 11월 프랜차이즈 정보공개서 신규등록 170개 브랜드 공개,http://www.shinailbo.co.kr/news/articleView.ht...
147,한시련 화면해설방송 주간 편성안내(12월11일~12월17일),https://www.beminor.com/news/articleView.html?...
148,한시련 제작 화면해설방송 주간 편성안내(12월11일~12월17일),https://www.ablenews.co.kr/news/articleView.ht...


In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import os

# 사용자로부터 시작 날짜와 종료 날짜 입력 받기
start_date_str = input('시작 날짜를 입력하세요 (YYYY-MM-DD): ')
end_date_str = input('종료 날짜를 입력하세요 (YYYY-MM-DD): ')

# 날짜 형식을 확인하고 datetime으로 변환
try:
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
except ValueError:
    print('올바른 날짜 형식이 아닙니다. YYYY-MM-DD 형식으로 입력해주세요.')
    exit()

# 시작 날짜와 종료 날짜 사이의 날짜 목록 생성
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# 결과를 저장할 딕셔너리 초기화
news_dict = {}
idx = 0

for date in date_list:
    date_str = date.strftime('%Y-%m-%d')

    # 검색할 쿼리와 네이버 뉴스 URL 생성
    query = f'{date_str} {query}'  # 날짜를 검색 쿼리에 포함
    query = query.replace(' ', '+')
    news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}'

    req = requests.get(news_url)
    soup = BeautifulSoup(req.text, 'html.parser')

    # 네이버 뉴스 크롤링
    table = soup.find('ul', {'class': 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

    for n in a_list:
        news_dict[idx] = {'title': n.get('title'), 'url': n.get('href')}
        idx += 1

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T
news_df

시작 날짜를 입력하세요 (YYYY-MM-DD): 2023-12-16
종료 날짜를 입력하세요 (YYYY-MM-DD): 2023-12-19
크롤링 완료
데이터프레임 변환


,title,url
0,"[데일리 IT 단신] SKT, 양자내성암호 5G망 연동 성공... 보안성 높인다 外",https://biz.newdaily.co.kr/site/data/html/2023...
1,"2023년 순천, 잡월드 농성부터 동부청사 개청까지[판읽기]",https://www.nocutnews.co.kr/news/6065230
2,2023 올해를 빛낸 스타는?...마동석·유재석·남궁민·뉴진스·임영웅 활약,http://www.wolyo.co.kr/news/articleView.html?i...
3,"[2023 ICT 10대 뉴스] 글로벌 불확실성 지속, 신사업 개화… '인적쇄신·경...",https://biz.newdaily.co.kr/site/data/html/2023...
4,[핀테크핫이슈] 역대 최대 소비자경보 발령...금융사기 '주의보',http://www.digitaltoday.co.kr/news/articleView...
5,"[유통가 레이더]""영패션관 리뉴얼 했더니""... 신세계 강남점, 2030 쇼핑 성지...",https://www.greened.kr/news/articleView.html?i...
6,2023 대중문화 키워드…SM·무빙·슬램덩크·서울의 봄,https://www.khan.co.kr/culture/culture-general...
7,활력 넘치는 서산시 건설 위한 다양한 활동 추진,https://www.joongdo.co.kr/web/view.php?key=202...
8,[위클립] 12월 3주,http://www.laborplus.co.kr/news/articleView.ht...
9,[브리프] 넷마블 넥슨 컴투스 카카오게임즈 스마일게이트 위메이드,http://www.biztribune.co.kr/news/articleView.h...


In [ ]:
!pip install --upgrade requests

In [ ]:
response = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90")
html = response.text
soup = BeautifulSoup(html, "html.parser")
articles = soup.select("div.info_group")

for article in articles:
    links = article.select("a.info")
    if len(links) >= 2:
        url = links[1].attrs["href"]
        response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})      # to avoid error use headers
        html = response.text                                                    # for each url get html
        soup = BeautifulSoup(html, "html.parser")                               # for each html make soup
        content = soup.select_one("#dic_area")                                  # get the body
        print(content.text)
        time.sleep(0.3)

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import os

# 사용자로부터 시작 날짜와 종료 날짜 입력 받기
start_date_str = input('시작 날짜를 입력하세요 (YYYY-MM-DD): ')
end_date_str = input('종료 날짜를 입력하세요 (YYYY-MM-DD): ')

# 날짜 형식을 확인하고 datetime으로 변환
try:
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
except ValueError:
    print('올바른 날짜 형식이 아닙니다. YYYY-MM-DD 형식으로 입력해주세요.')
    exit()

# 시작 날짜와 종료 날짜 사이의 날짜 목록 생성
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# 결과를 저장할 딕셔너리 초기화
news_dict = {}
idx = 0

for date in date_list:
    date_str = date.strftime('%Y-%m-%d')

    # 검색할 쿼리와 네이버 뉴스 URL 생성
    query = f'{date_str} {query}'  # 날짜를 검색 쿼리에 포함
    query = query.replace(' ', '+')
    news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}'

    req = requests.get(news_url)
    soup = BeautifulSoup(req.text, 'html.parser')

    # 네이버 뉴스 크롤링
    table = soup.find('ul', {'class': 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

    for n in a_list:
        news_dict[idx] = {'title': n.get('title'), 'url': n.get('href')}
        idx += 1

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T
news_df

## 원래처럼 네이버 뉴스 검색

In [ ]:
if __name__ == "__main__":
    query = input('뉴스를 검색할 키워드를 입력하세요: ')
    start_date = input('검색 시작일(YYYY-MM-DD)을 입력하세요: ')
    end_date = input('검색 종료일(YYYY-MM-DD)을 입력하세요: ')

    news_dict = crawl_naver_news(query, start_date, end_date)

    # 데이터프레임 변환
    news_df = pd.DataFrame(news_dict).T

    # 현재 작업 디렉토리에 엑셀 파일 저장
    folder_path = os.getcwd()
    xlsx_file_name = '네이버뉴스_{}_{}_to_{}.xlsx'.format(query, start_date, end_date)
    news_df.to_excel(xlsx_file_name, index=False)

In [ ]:
2# 네이버 검색 API 호출 URL
api_url = "https://openapi.naver.com/v1/search/news.json"

client_id = 'Zwdz_lbupo91QAvrtqsz'
client_secret = 'ifGZTCfREm'

# API 호출 시 사용할 헤더 정보 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

In [ ]:
df['date'] = pd.to_datetime(df['date'])

# 날짜를 기준으로 내림차순 정렬
df = df.sort_values(by='date', ascending=False)
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   press   100 non-null    object
 1   date    100 non-null    object
 2   title   100 non-null    object
 3   link    100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [ ]:
!pip install Kiwipiepy

from kiwipiepy import Kiwi
import pandas as pd

kiwi = Kiwi()

# 품사 필터링 및 중복 제거 함수
def filter_pos(text, target_pos=['NNG']):        # 동사 뺴고 명사만 필터링하자(변경사항)
    tokens = kiwi.analyze(text)[0][0]
    filtered_tokens = [word[0] for word in tokens if word[1] in target_pos]

    # 중복 제거 (순서 유지)
    unique_tokens = []
    for token in filtered_tokens:
        if token not in unique_tokens:
            unique_tokens.append(token)

    return unique_tokens

# '기사 제목' 열에 대해 형태소 분석 및 중복 제거하여 '형태소' 열에 추가
data_df['제목 키워드'] = data_df['제목'].apply(filter_pos)
titles = data_df['제목'].tolist()

In [ ]:
DF = df_sample.copy()
DF

,번호,제목,작성자,작성일,조회,제목 키워드,기사 개수,url
0,5097,"대법 ""장기요양 판정 전 사망하면 진단보험금 수령 불가""",공보관실,2023-11-03,62,"[대법, 장기, 요양, 판정, 전, 사망, 진단, 보험금, 수령, 불가]",4,https://news.bbsi.co.kr/news/articleView.html?...
1,5096,"검사 실수로 동명이인에게 벌금형...대법 ""공소 기각""(종합)",공보관실,2023-11-03,58,"[검사, 실수, 이인, 벌금형, 대법, 공소, 기각, 종합]",1,https://n.news.naver.com/mnews/article/003/001...
2,5095,"기관 흡인 후 저산소증으로 숨진 신생아···대법 ""병원 과실 단정 못해""",공보관실,2023-10-31,204,"[기관, 흡인, 후, 산소증, 신생아, 대법, 병원, 과실, 단정]",1,http://www.newspim.com/news/view/20231027000942
3,5094,"사무실서 상사 대화 내용 몰래 녹음한 공무원...대법 ""유죄""",공보관실,2023-10-31,187,"[사무실, 서, 상사, 대화, 내용, 녹음, 공무원, 대법, 유죄]",2,https://n.news.naver.com/mnews/article/214/000...
4,5093,'선거구민 음식 제공' 박정희 청주시의원 당선무효형 확정,공보관실,2023-10-31,78,"[선거, 구민, 음식, 제공, 의원, 당선, 무효형, 확정]",80,https://n.news.naver.com/mnews/article/056/001...
...,...,...,...,...,...,...,...,...
195,4902,"""취업규칙에 없는 탄력 근로제 도입은 부당""",공보관실,2023-05-15,517,"[탄력, 근로, 도입, 부당]",6,https://n.news.naver.com/mnews/article/050/000...
196,4901,"'주문 실수' 2분 만에 460억 날린 한맥 사건···한국거래소, 9년 ...",공보관실,2023-05-15,557,"[주문, 실수, 맥, 사건]",1,https://n.news.naver.com/mnews/article/032/000...
197,4900,"한국 회사 땅 침범한 몽골대사관...대법 ""금전지급 청구 가능""",공보관실,2023-05-12,546,"[회사, 땅, 침범, 대사관, 대법, 금전, 지급, 청구, 가능]",1,https://n.news.naver.com/mnews/article/421/000...
198,4899,"대법 ""석면폐증, 진폐증과 유사...장해급여 바로 지급해야""",공보관실,2023-05-12,382,"[대법, 석면폐증, 진폐증, 유사, 장해, 급여, 지급]",1,https://n.news.naver.com/mnews/article/003/001...


In [ ]:
DF.to_excel('example.xlsx', sheet_name='MySheet', index=False)

# 저장된 파일 확인
import os
os.listdir()  # 현재 작업 디렉토리에 저장된 파일 목록을 확인

['.config', 'drive', 'example.xlsx', 'sample_data']